In [ ]:
import os,sys
import kfp
from dkube.pipelines import dkube_training_op, dkube_serving_op
from dkube.sdk import DkubeApi, DkubeCode, DkubeDataset, DkubeModel, DkubeTraining, DkubeServing

In [ ]:
username = os.getenv("USERNAME")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)
api = DkubeApi(token=token)

In [ ]:
code_name = "dkube-examples"
model_name = "mnist"
dataset_name = "mnist"
image = "ocdr/dkube-datascience-tf-cpu:v2.0.0"

In [ ]:
#Create code repo 
code = DkubeCode(username,code_name)
code.update_git_details(url="https://github.com/oneconvergence/dkube-examples-2.0.git", branch="tensorflow")
#api.create_code(code)

#Create dataset repo
dataset = DkubeDataset(username,dataset_name)
dataset.update_git_details(url="https://dkube.s3.amazonaws.com/datasets/mnist.pkl.gz")
dataset.update_dataset_source(source="pub_url")
#api.create_dataset(dataset)

#Create model repo
model = DkubeModel(username, model_name)
#api.create_model(model)

In [ ]:
training = DkubeTraining(username, name="train")
training.update_container(framework="tensorflow_2.0.0", image_url=image)
training.add_code(code_name, commitid="")
training.update_startupscript("python mnist/train.py")
training.add_input_dataset(dataset_name, mountpath='/mnist')
training.add_output_model(model_name, mountpath='/opt/dkube/output')
training.add_envvar("EPOCH","1")

serving = DkubeServing(username, name="serving")
serving.set_transformer(True, script='mnist/transformer.py')
serving.update_serving_model(model_name)
    
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(token, program='training'):
    train   = dkube_training_op("train", authtoken=token, training=training)    
    dkube_serving_op("serving", authtoken=token, serving=serving).after(train)
    

In [ ]:
client.create_run_from_pipeline_func(mnist_pipeline, arguments={"token":token})

In [ ]:
#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline.zip")
client.upload_pipeline("mnist-pipeline.zip")